In [2]:
import numpy as np
import pandas as pd
import pickle

In [3]:
def load_pkl(fname):
    with open(fname, 'rb') as infile:
        return pickle.load(infile)
def save_pkl(df, fname):
    with open(fname, 'wb') as outfile:
        pickle.dump(df, outfile)
def na_percent(df):
    return df.isna().sum()/len(df)
def map_customer_id(df):
    df['customer_id'] = df['customer_id'].apply(lambda x: int(x[-16:],16) ).astype('int64')
def map_article_id(df):
    df['article_id'] = df['article_id'].astype('int32')
    # when making predictions, should convert to string and add leading 0
    # e.g. train['article_id'] = '0' + train.article_id.astype('str')
path='/content/drive/MyDrive/Colab Notebooks/'

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Baseline Recommendation

Recommend to each customer their most frequently bought products in the past 6 months (up to 12 products).

If < 12 recomendations, fill in with most popular products.

mAP 0.0068

In [5]:
df_transactions_train = load_pkl(path+'df_transactions_train.pkl')
df_transactions_train["day_num"] = df_transactions_train["year"]*365 + df_transactions_train["month"]*30 + df_transactions_train["day"]
df_transactions_train = df_transactions_train[df_transactions_train["day_num"]>max(df_transactions_train["day_num"])-180]

In [6]:
count_group = df_transactions_train[['customer_id', 'article_id', 'day_num']].groupby(['customer_id', 'article_id']).count()
count_group.rename(columns={'day_num':'count'}, inplace=True)

In [8]:
df_transactions_train = pd.merge(df_transactions_train, count_group, on=['customer_id', 'article_id'], how='left')

In [9]:
X_train_group = df_transactions_train[['customer_id', 'article_id', 'count']].sort_values(['customer_id', 'count'],ascending=False).groupby('customer_id').head(12)
X_train_group = X_train_group.groupby('customer_id')['article_id'].apply(list).reset_index(name='prediction2')
X_train_group['prediction2'] = X_train_group['prediction2'].apply(lambda x: '0'+' 0'.join([str(a) for a in x]))

In [10]:
df_sample_submission = load_pkl(path+'df_sample_submission.pkl')

In [11]:
df_sample_submission = pd.merge(df_sample_submission, X_train_group, on='customer_id', how='left')
df_sample_submission['prediction2'] = df_sample_submission['prediction2'].fillna('')

In [12]:
df_sample_submission['prediction'] = df_sample_submission.apply(lambda row: (row['prediction2']+row['prediction'])[:131], axis=1)
df_sample_submission.drop(columns=['prediction2'], inplace=True)

In [13]:
df_sample_submission

,customer_id,prediction
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0841260003 0887593002 0890498002 0795440001 08...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0811835004 0811835004 0811835004 0811835004 05...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0351484002 0351484002 0750424014 0750424014 08...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,0706016001 0706016002 0372860001 0610776002 07...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,0896152002 0730683050 0927530004 0791587015070...
...,...,...
1371975,ffffbbf78b6eaac697a8a5dfbfd2bfa8113ee5b403e474...,0826955010 0826955010 0557599022 0557599022 08...
1371976,ffffcd5046a6143d29a04fb8c424ce494a76e5cdf4fab5...,0699623004 0699623004 0663568009 0663568009 07...
1371977,ffffcf35913a0bee60e8741cb2b4e78b8a98ee5ff2e6a1...,0706016002 0689365050 0884081001 0794819001 07...
1371978,ffffd7744cebcf3aca44ae7049d2a94b87074c3d4ffe38...,0866755002 0866755002 0821395003 0806241002 07...


In [14]:
import lightgbm
df_sample_submission.to_csv(path+'submission_baseline.csv', index=False)